# Create Scheduled Automations (Arrears)

## Caveats:
Using the SQL option (as would be expected on first time setup), the user cannot construct a valid CSV. This will create a CSV copy of a given table (name) - not what is needed! Ideally this should construct a table with the (valid) columns:
  
    CustomerId,Username,Pw,MessageTemplateId,AutomationId

Currently, it is preferred to input as CSV (choose 'n' on step 2.) and have a pre-valdiated Message Template ID, CustomerIDs, etc...

## Prerequisites:
--------------------

For the sake of simplicity and faster development, each customer: 
*  **SHOULD NOT HAVE AN EXISTING AUTOMATION YET**.
   * Otherwise an automation will be inserted but the updating of the automation with a schedule may not occur for the one created.
* have access to a valid messageschedule (template in UI) **or** access to a messagescheduledefault (these work for all customers).
   * This is set in the CSV data read-in (or SQL).

## Workflow:
--------------------
1. Set Constants;
2. Ask whether to read from db;
   * (yes) Input csv file name to export data to and eventually also read from;
     * Add db credentials and server path;
     * Export read data to a csv file.
   * (no)  Input csv file name read from (previously exported to from other scripts).
     * Read from csv file and load customer data into memory.
3. Configure API Requests;
   * Authenticate customers.
4. Configure Automation Rules & Schedule;

**Note:** (*Steps 5, 6, and 7* done separately as the current arrears automation UI process does these separately and there is currently no endpoint that inserts with schedule. Inserting also does not return an id).

5. Insert automations per Customer session;
6. Query Automations per customer and obtain their Automation ID for the first Arrears Automation found;
7. Update all Automations with a Schedule (configured in step 4).

~~**Note: Current caveat --> the same message template is being used for all rules.**~~

**Note: A default message template is assigned to all Automations.** This should be fine, as the `messsagescheduledefault` schema is not customerContext bound.


In [1]:
import pandas as pd
from requests import Session
from tqdm.auto import tqdm

from Auth import UserSessionsHandler
from CommonLib import NotebookHelper, ConsoleHelpers
from CommonLib.ListHelpers import FilterList
from CommonLib.ResponseDecorators import PropertyMeHttpRequestExceptionHandler
from Config import (ConfigureCsvFilename, ConfigurePropertyMeBaseUrl,
                    ConfigureUrl, ConfigureMySqlConnectionString,
                    ConfigureDbTableName)
from Database import *

KeyboardInterrupt: 

## 1. Set Constants
--------------------

In [ ]:
baseUrl = ConfigurePropertyMeBaseUrl()
queryAutomationUrl = baseUrl + '/api/automation/automations?Type=Arrears'
createAutomationUrl = baseUrl + '/api/automation/automations'
patchAutomationEndpointUrl = baseUrl + '/api/automation/automations'

divider = "********************************************************************************"
createAutomationData = { "Type": "Arrears", "Name": "Load Test Automation", "IsActive": True }

fullFilenameDefault = '2021-10-25_DEV2_deepuser-data'
fullFilename = None
dbString = None

# MemberId isnt needed in this workflow, but keeping it here to not overwrite old data
customers = pd.DataFrame()

## 2. Set Datasource
--------------------

In [ ]:
# Retrieve the load testing customer data
print(divider)

csvExplanation = """
    The csv will require columns titled: CustomerId, Username, Pw, MessageTemplateId
    If these values are missing, then subsequent operations may fail unexpectedly.
    """
print("\n== CONFIGURING CSV ==")
print(csvExplanation)

fullFilename = ConfigureCsvFilename(fullFilenameDefault)

# Read from csv (expecting 4 columns)
print('\nREADING FROM CSV TO LOAD CUSTOMER DATA:')
customers = pd.read_csv(fullFilename)
print('\nCustomer rows from file (customerId, username, messageTemplateId, memberId, managerId) (Password not shown):')
NotebookHelper.Output(customers.drop(["Pw"], axis=1).head())

print(divider)

In [5]:
# the wide test user
wideUser = (pd
    .read_csv(fullFilename)
    .drop(["Unnamed: 0"], axis=1, errors='ignore')
)

# propogate customerWide across the all of the wide automations.
customerWide = (pd
    .read_csv(ConfigureCsvFilename("deepuser-aa-load-test-inserted-automations-for-{CUSTOMER_ID_HERE}"))
    .assign(CustomerId=wideUser.CustomerId.values[0])
    .assign(Username=wideUser.Username.values[0])
    .assign(Pw=wideUser.Pw.values[0])
    .assign(MessageTemplateId=wideUser.MessageTemplateId.values[0])
    .assign(MemberId=wideUser.MemberId.values[0])
)

Enter the csv filename (exclude the .csv extension suffix. Enter no value to default to "deepuser-aa-load-test-inserted-automations-for-{CUSTOMER_ID_HERE}"):  2021-10-25_DEV2_wide-automations-for-adcc00f4-f7b3-4079-a1aa-250e7e864504



The full filename will be "2021-10-25_DEV2_wide-automations-for-adcc00f4-f7b3-4079-a1aa-250e7e864504.csv"


## 3. Configure API Requests
--------------------

In [6]:
# Configure the load testing 'customer' authentication sessions.
print(divider)
print('\nAUTHENTICATING AND OBTAINING SESSION DATA FOR EACH CUSTOMER:')

def MakeSession(s: pd.Series, handler: UserSessionsHandler):
    """Functional approach is a neat way to easily auth users in a dataframe."""
    return handler.AddUserSession(s.Username, s.Pw, customerId=s.CustomerId)

pmeHandler = UserSessionsHandler(baseUrl)
customers.apply(lambda c: MakeSession(c, pmeHandler), axis=1)
pmeHandler.AuthenticateSessions()

print('\n\n**CHECK CUSTOMER DATA IS ALL EXPECTED**')
print(f"{len(pmeHandler.Sessions)} sessions were created.")
print(f"Session summary:\t{pmeHandler.Sessions[:5]}")
endSectionMessage = "To re-execute the script and start again, close the program."
ConsoleHelpers.PreventImmediateConsoleClose(endSectionMessage)

print(divider)

********************************************************************************

AUTHENTICATING AND OBTAINING SESSION DATA FOR EACH CUSTOMER:
Authenticating for deeptest_aaloadtestinguser@propertyme.com


**CHECK CUSTOMER DATA IS ALL EXPECTED**
1 sessions were created.
Session summary:	[Login: deeptest_aaloadtestinguser@propertyme.com | IsAuthenticated: True]


To re-execute the script and start again, close the program.


Hit "Enter" with no value to close console. 


********************************************************************************


## 4. Configure Automation Rules & Schedule
------------------

In [16]:
# Setup automation rules and schedules
print(divider)
print('\nCONFIGURE AUTOMATION RULES AND SCHEDULES:')

# Setup weekdays for schedule to occur
while True:
    useAllWeekdays = input('Do you want to default to all business days of week in the schedule (Monday - Friday)? (Type "y" or "n"): ')
    if (useAllWeekdays == 'y' or useAllWeekdays == 'n'):
        break

daysOfWeek = []
    
if (useAllWeekdays == 'n'):
    listOfDays = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
    print('Accepted days: "Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"')
    
    while True:
        print(f'Current selected days of week {daysOfWeek}')
        dayEntered = input('Type in a day or hit "Enter" immediately to proceed with selected days of week. (Finishing with empty list defaults to all business days): ')
        
        if not dayEntered:
            break 
        
        if dayEntered in listOfDays and dayEntered not in daysOfWeek:
            daysOfWeek.append(dayEntered)
        

if len(daysOfWeek) == 0:
    daysOfWeek = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

print(f'\nSelected days of week: {daysOfWeek}.\n')

# Setup the time for schedule to occur on each selected day
print('Minutes into the days reference: 540 = 9am, 600 = 10am, 660 = 11am, 720 = 12pm, 780 = 1pm, 840 = 2pm, 900 = 3pm, 960 = 4pm, 1020 = 5pm.')
while True:
    minutesInput = input('Enter the minutes into the day for each automation to be scheduled at: ')
    try:
        minutesIntoDays = int(minutesInput)
        if (minutesIntoDays >= 0 and minutesIntoDays < 1440):
            break 
    except ValueError:
        print(f'{minutesInput} is not a valid value.')
    print(f'Please enter an integer from 0 - 1439.')
        
print(f'\nSelected minutes into days: {minutesIntoDays}.\n')
    
# Setup days in arrears for rent (one condition per rule)
print('By default all automations will be updated with rules for 3, 5 and 7 days in rent arrears and all with the same messageTemplateId for that customer (as read from csv file).')
while True:
    useDefaultRules = input('Do you want to use the default? (Type "y" or hit "Enter" immediately to use defaults. Enter "n" to define your own days): ')
    if not useDefaultRules or useDefaultRules == 'y' or useDefaultRules == 'n':
        break 
        
daysInArrears = []

if (useDefaultRules == 'n'):
    while True:
        print(f'Current days in arrears {daysInArrears}')
        daysInArrearsInput = input('Type in a number or hit "Enter" immediately to proceed with selected days in arrears values. (Empty list reverts to using default values): ')
        
        if not daysInArrearsInput:
            break 
            
        try:
            daysInArrearsInt = int(daysInArrearsInput)
            if daysInArrearsInt not in daysInArrears:
                daysInArrears.append(daysInArrearsInt)
        except ValueError:
            print(f'{daysInArrearsInput} is not a valid integer value')
    
if len(daysInArrears) == 0:
    daysInArrears = [3, 5, 7]
else:
    daysInArrears = sorted(daysInArrears)

scheduleSummary = f"""SELECTED VALUES:
    Days of week: {daysOfWeek}
    Minutes into days: {minutesIntoDays}
    Days in arrears: {daysInArrears}
    """.format(daysOfWeek, minutesIntoDays, daysInArrears)

print('\n**CHECK VALUES ARE EXPECTED**')
print(scheduleSummary)
input('Hit "Enter" to continue to authenticating each customer. To re-execute the script and start again, close the program.') 
print(divider)

********************************************************************************

CONFIGURE AUTOMATION RULES AND SCHEDULES:


Do you want to default to all business days of week in the schedule (Monday - Friday)? (Type "y" or "n"):  y



Selected days of week: ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'].

Minutes into the days reference: 540 = 9am, 600 = 10am, 660 = 11am, 720 = 12pm, 780 = 1pm, 840 = 2pm, 900 = 3pm, 960 = 4pm, 1020 = 5pm.


Enter the minutes into the day for each automation to be scheduled at:  975



Selected minutes into days: 975.

By default all automations will be updated with rules for 3, 5 and 7 days in rent arrears and all with the same messageTemplateId for that customer (as read from csv file).


Do you want to use the default? (Type "y" or hit "Enter" immediately to use defaults. Enter "n" to define your own days):  



**CHECK VALUES ARE EXPECTED**
SELECTED VALUES:
    Days of week: ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    Minutes into days: 975
    Days in arrears: [3, 5, 7]
    


Hit "Enter" to continue to authenticating each customer. To re-execute the script and start again, close the program. 


********************************************************************************


## 7. Update all Automations with a Schedule (configured in step 4)
----------------

In [18]:
# Patch automations with a schedule
print(divider)
print('\nPATCHING EACH AUTOMATION WITH A SCHEDULE:')
    
def ConstructCondition(daysInArrears: int):
    return {
        "Type": "TenantArrears", 
        "ArrearsType": "rent", 
        "DaysInArrears": daysInArrears
    }

def ConstructAction(templateId: str):
    return {
        "Type": "SendTemplateMessage",
        "MessageTemplateId": templateId
    }

def ConstructRule(daysInArrears: int, templateId: str):
    return {
        "Conditions": [ConstructCondition(daysInArrears)],
        "Actions": [ConstructAction(templateId)]
    }

def ConstructRules(daysInArrearsList: list, templateId: str):
    rules = [] 
    for daysInArrearsNum in daysInArrearsList:
        rules.append(ConstructRule(daysInArrearsNum, templateId))
    return rules 

@PropertyMeHttpRequestExceptionHandler(True)
def PatchAutomationRequest(requestSession: Session, automationId: str, json: dict):
    global patchAutomationEndpointUrl
    return requestSession.patch(f'{patchAutomationEndpointUrl}/{automationId}', json=json)

def PatchAutomation(requestSession: Session, automationId: str, templateId: str):
    patchAutomationData = {
        "Name": "Load test Automation",
        "Rules": ConstructRules(daysInArrears, templateId),
        "Schedule": {
            "DaysOfWeek": daysOfWeek,
            "MinutesIntoDays": minutesIntoDays
        }
    }
    
    patchResp = PatchAutomationRequest(requestSession, automationId, patchAutomationData)
    
    if patchResp is not None:
        print(f"Automation PATCH for ID: '{automationId}'")
        
# functional approach is a neat way to easily propogate PATCHs for users in a dataframe
def PatchAuto(s: pd.Series):
    global pmeHandler
    session = pmeHandler.Sessions[0].Current
    PatchAutomation(session, s.AutomationId, s.MessageTemplateId)
    return

print(f"PATCH requests to be made to base url '{patchAutomationEndpointUrl}/<automationId>'")
print(f"Preparing to update automations with schedules for {customers.shape[0]} customers...")

customerWide.apply(PatchAuto, axis=1)

print(divider)
ConsoleHelpers.PreventImmediateConsoleClose('Completed. Hit "Enter" immediately to close console.')

********************************************************************************

PATCHING EACH AUTOMATION WITH A SCHEDULE:
PATCH requests to be made to base url 'https://app-dev2.sandbox.propertyme.com/api/automation/automations/<automationId>'
Preparing to update automations with schedules for 1 customers...
Automation PATCH for ID: '1f8ad0bb-0e06-49c8-86c6-8897ec946bfd'
Automation PATCH for ID: 'f906b80b-1cbf-4c77-972a-fc6c7e155d71'
Automation PATCH for ID: '549e18f8-bdf4-4a55-97e6-6157a9052a84'
Automation PATCH for ID: 'e6d5a412-91ec-4f3c-a787-c167a5b545fb'
Automation PATCH for ID: '78b26f7d-3f5e-4e93-aefb-9d811ed277a3'
Automation PATCH for ID: '000596c7-9a7e-47e6-afeb-3b4879d083e3'
Automation PATCH for ID: '924b8bb7-e0d8-4379-bae1-774d0ed6e1a6'
Automation PATCH for ID: '911eb1ed-4161-4dab-92b8-b2173f7cc228'
Automation PATCH for ID: '84c1b16e-1d5c-4a6f-a165-11d702a10a25'
Automation PATCH for ID: 'cf00d6d4-b4d8-4f55-a2cb-5998160b26bc'
Automation PATCH for ID: '58e3c809-a855-4cdd-a8c

Hit "Enter" with no value to close console. 
